## SRJ - Exercise

In [1]:
#Adapt VQE 
#ash_excitation= [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [3, 7], [2, 6], [2, 3, 6, 7], [1, 5], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 4, 5], [1, 2, 5, 6], [0, 3, 4, 7], [0, 2, 4, 6], [1, 3, 5, 7], [0, 3, 5, 6], [3, 7], [2, 6], [0, 2, 4, 6], [1, 5], [0, 3, 5, 6], [2, 3, 6, 7], [2, 3, 6, 7], [2, 3, 6, 7]]
#energies =  [-1.4300836708403637, -1.5694603835521226, -1.743633442615135, -1.8421101514364198, -1.8504359363287717, -1.859460025024687, -1.8656767410818902, -1.8663879748307473, -1.8843560329822533, -1.884436165409815, -1.8844361671706917, -1.8844364522100128, -1.8844364556672768, -1.8844365760136006, -1.88443657646997, -1.8844612572154111, -1.8850966647715137, -1.8850974254839188, -1.885097434518275, -1.885124904085803, -1.8851249040961857, -1.8851249040964084, -1.885124904096495, -1.8851249040964824, -1.8851249040964708]


In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        print('ashex:', ash_excitation)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[0], ash_excitation[1] + 1)), wires2=list(range(ash_excitation[2], ash_excitation[3] + 1)))
            elif len(ash_excitation) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[0], ash_excitation[1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[0], ash_excitation[1] + 1)), wires2=list(range(ash_excitation[2], ash_excitation[3] + 1)))
            elif len(ash_excitation) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[0], ash_excitation[1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation[-1], hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        print('The params are', params)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation[-1], params[-1])
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation[-1],params[-1]))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=3) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
-1.1078408262252637 * I(0) + 0.050636412339760725 * Z(0) + 0.00202295733102974 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.00202295733102974 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04498894793099842 * Z(2) + 0.02579174220115818 * (Z(0) @ Z(2)) + 0.0021181739685781284 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0021181739685781284 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.0062387146111867135 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.0062387146111867135 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.038410202743309765 * Z(4) + 0.03299377398000802 * (Z(0) @ Z(4)) + 0.032312626825255715 * Z(6) + 0.03866342276424449 * (Z(0) @ Z(6)) + 0.050636412339760704 * Z(1) + 0.07316430147427193 * (Z(0) @ Z(1)) + 0.00889991182487373 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.00889991182487373 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.045126677014678436 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045126677014678436 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.

IndexError: index 1 is out of bounds for axis 0 with size 1

In [4]:
ash_excitation[-1]

2

## 4 states - lets see the subspace

In [3]:

ash_excitation= [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7]]
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5]]
#ash_excitation = [[2,3,6,7]]
#params = [0.82882176, 1.28813274 ,1.0080735,  1.1638334 ]
params = qml.numpy.array([np.pi/4, np.pi/4, np.pi/4, np.pi/4])  #Example params, can be changed
#params = [0.82882176]
hf_state = np.array([1,1,1,1,0,0,0,0])
qubits = 8
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def final_energy(params, ash_excitation, hf_state, H):
    print('ash_excitation', ash_excitation) 
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #print('DE', ash_excitation[i])
            #print('Params', params[i])  
            qml.DoubleExcitation(params[i], wires = excitation)
        elif len(ash_excitation[i]) == 2:
            qml.SingleExcitation(params[i], wires = excitation)
    return qml.state()
dev1 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
@qml.qnode(dev1)
def measure(ostate):
    qml.StatePrep(ostate, wires=range(qubits))
    return qml.counts()

final_energy_value = final_energy(params, ash_excitation,hf_state, H)
counts = measure(final_energy_value)
print('Counts=', counts)
print('counts lenghth=', len(counts))


ash_excitation [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7]]
Counts= {'00001111': 70, '00111100': 112, '01100110': 68, '10011001': 84, '11000011': 110, '11110000': 556}
counts lenghth= 6


In [4]:
len(counts)

6

## 4 Individual states

In [8]:
states_sum= []
finalcounts = []
ash_excitation= [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0,4]]
#ash_excitation = [[2,3,6,7]]
params = [0.82882176, 1.28813274 ,1.0080735,  1.1638334, 0.785 ]
#params = [0.82882176]
hf_state = np.array([1,1,1,1,0,0,0,0])
qubits = 8
dev = qml.device("lightning.qubit", wires=qubits)

for i, excitation in enumerate(ash_excitation):
    @qml.qnode(dev)
    def ind_energy(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        if len(ash_excitation[i]) == 4:
            print('DE', ash_excitation[i])
            print('Params', params[i])  
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('SE', ash_excitation[i])
            print('Params', params[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
        return qml.state()
    states_sum.append(ind_energy(params, ash_excitation, hf_state, H))

dev1 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
@qml.qnode(dev1)
def measure(ostate):
    qml.StatePrep(ostate, wires=range(qubits))
    return qml.counts()

for i, state in enumerate(states_sum):
    counts = measure(state)
    print(f"Counts for excitation {ash_excitation[i]}:", counts)
    finalcounts.append(counts)
print(len(finalcounts))

DE [2, 3, 6, 7]
Params 0.82882176
DE [0, 3, 5, 6]
Params 1.28813274
DE [0, 1, 4, 5]
Params 1.0080735
DE [1, 2, 4, 7]
Params 1.1638334
SE [0, 4]
Params 0.785
Counts for excitation [2, 3, 6, 7]: {'11000011': 160, '11110000': 840}
Counts for excitation [0, 3, 5, 6]: {'01100110': 389, '11110000': 611}
Counts for excitation [0, 1, 4, 5]: {'00111100': 241, '11110000': 759}
Counts for excitation [1, 2, 4, 7]: {'10011001': 305, '11110000': 695}
Counts for excitation [0, 4]: {'01111000': 149, '11110000': 851}
5


## 5 states

In [6]:
ash_excitation= [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0,4]]
params = qml.numpy.array([np.pi/4, np.pi/4, np.pi/4, np.pi/4, np.pi/4])  #Example params, can be changed

final_energy_value = final_energy(params, ash_excitation,hf_state, H)
counts = measure(final_energy_value)
print('Counts=', counts)
print('counts lenghth=', len(counts))

#Individual state measurements
states_sum.append(ind_energy(params, ash_excitation, hf_state, H))
print(len(states_sum))
for i, state in enumerate(states_sum):
    counts = measure(state)
    print(f"Counts for excitation {ash_excitation[i]}:", counts)

ash_excitation [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4]]
Counts= {'00001111': 71, '00111100': 91, '01001011': 18, '01100110': 75, '01111000': 80, '10000111': 12, '10011001': 89, '10110100': 16, '11000011': 104, '11110000': 444}
counts lenghth= 10
SE [0, 4]
Params 0.7853981633974483
6


Counts for excitation [2, 3, 6, 7]: {'11000011': 165, '11110000': 835}
Counts for excitation [0, 3, 5, 6]: {'01100110': 378, '11110000': 622}
Counts for excitation [0, 1, 4, 5]: {'00111100': 235, '11110000': 765}
Counts for excitation [1, 2, 4, 7]: {'10011001': 284, '11110000': 716}
Counts for excitation [0, 4]: {'01111000': 148, '11110000': 852}


IndexError: list index out of range

In [2]:
H = (-1.1078408262252637 * I(0) + 0.050636412339760725 * Z(0) + 0.00202295733102974 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.00202295733102974 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04498894793099842 * Z(2) + 0.02579174220115818 * (Z(0) @ Z(2)) + 0.0021181739685781284 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0021181739685781284 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.0062387146111867135 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.0062387146111867135 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.038410202743309765 * Z(4) + 0.03299377398000802 * (Z(0) @ Z(4)) + 0.032312626825255715 * Z(6) + 0.03866342276424449 * (Z(0) @ Z(6)) + 0.050636412339760704 * Z(1) + 0.07316430147427193 * (Z(0) @ Z(1)) + 0.00889991182487373 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.00889991182487373 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.045126677014678436 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045126677014678436 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045126677014678436 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045126677014678436 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.002930283785050422 * (Y(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Y(6)) + -0.002930283785050422 * (Y(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Y(6)) + -0.002930283785050422 * (X(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ X(6)) + -0.002930283785050422 * (X(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ X(6)) + 0.002022957331029742 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + 0.00889991182487373 * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + 0.002022957331029742 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.00889991182487373 * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.038218440738223496 * (Y(0) @ X(1) @ X(4) @ Y(5)) + -0.038218440738223496 * (Y(0) @ Y(1) @ X(4) @ X(5)) + -0.038218440738223496 * (X(0) @ X(1) @ Y(4) @ Y(5)) + 0.038218440738223496 * (X(0) @ Y(1) @ Y(4) @ X(5)) + 0.002930283785050422 * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.002930283785050422 * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.002930283785050422 * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.002930283785050422 * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.03551335762260015 * (Y(0) @ X(1) @ X(6) @ Y(7)) + -0.03551335762260015 * (Y(0) @ Y(1) @ X(6) @ X(7)) + -0.03551335762260015 * (X(0) @ X(1) @ Y(6) @ Y(7)) + 0.03551335762260015 * (X(0) @ Y(1) @ Y(6) @ X(7)) + 0.008254330656463974 * (Y(0) @ Z(1) @ Z(3) @ Y(4)) + 0.008254330656463974 * (X(0) @ Z(1) @ Z(3) @ X(4)) + -0.0038180782116323244 * (Y(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Y(6)) + -0.011010996943064903 * (Y(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Y(6)) + -0.007192918731432575 * (Y(0) @ Z(1) @ Y(2) @ X(4) @ Z(5) @ X(6)) + -0.007192918731432575 * (X(0) @ Z(1) @ X(2) @ Y(4) @ Z(5) @ Y(6)) + -0.011010996943064903 * (X(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ X(6)) + -0.0038180782116323244 * (X(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ X(6)) + 0.04498894793099845 * Z(3) + 0.07091841921583662 * (Z(0) @ Z(3)) + -0.0018524415647159886 * (Y(0) @ Z(1) @ Z(2) @ Y(4)) + -0.0018524415647159886 * (X(0) @ Z(1) @ Z(2) @ X(4)) + 0.010106772221179965 * (Y(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ Y(2)) + 0.010106772221179965 * (Y(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ Y(2)) + 0.010106772221179965 * (X(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ X(2)) + 0.010106772221179965 * (X(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ X(2)) + 0.03465930335238148 * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Y(6)) + 0.03465930335238148 * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Y(6)) + 0.03465930335238148 * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ X(6)) + 0.03465930335238148 * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ X(6)) + 0.00211817396857813 * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.009168998396237136 * (Z(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.00211817396857813 * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.009168998396237136 * (Z(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + -0.038477381564013806 * (Y(0) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2)) + 0.038477381564013806 * (Y(0) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2)) + 0.038477381564013806 * (X(0) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2)) + -0.038477381564013806 * (X(0) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2)) + 0.006229806418892229 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(6)) + 0.006229806418892229 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(6)) + 0.03841020274330973 * Z(5) + 0.07121221471823151 * (Z(0) @ Z(5)) + -0.0021078400948430893 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(5)) + -0.0021078400948430893 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(5)) + -0.04567030029544638 * (Y(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ Y(2)) + -0.04567030029544638 * (Y(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ Y(2)) + -0.04567030029544638 * (X(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ X(2)) + -0.04567030029544638 * (X(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ X(2)) + -0.002927093988266862 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Y(7)) + 0.002927093988266862 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ X(7)) + 0.002927093988266862 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Y(7)) + -0.002927093988266862 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ X(7)) + 0.03231262682525571 * Z(7) + 0.07417678038684464 * (Z(0) @ Z(7)) + 0.00915690040715909 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(7)) + 0.00915690040715909 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(7)) + 0.02579174220115818 * (Z(1) @ Z(3)) + -0.0062387146111867135 * (Z(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + -0.0062387146111867135 * (Z(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.03299377398000802 * (Z(1) @ Z(5)) + 0.03866342276424449 * (Z(1) @ Z(7)) + 0.07091841921583662 * (Z(1) @ Z(2)) + -0.0018524415647159886 * (Y(1) @ Z(3) @ Z(4) @ Y(5)) + -0.0018524415647159886 * (X(1) @ Z(3) @ Z(4) @ X(5)) + 0.010106772221179963 * (Y(1) @ X(2) @ X(3) @ Y(4)) + -0.010106772221179963 * (Y(1) @ Y(2) @ X(3) @ X(4)) + -0.010106772221179963 * (X(1) @ X(2) @ Y(3) @ Y(4)) + 0.010106772221179963 * (X(1) @ Y(2) @ Y(3) @ X(4)) + 0.03465930335238148 * (Y(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Y(7)) + 0.03465930335238148 * (Y(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Y(7)) + 0.03465930335238148 * (X(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ X(7)) + 0.03465930335238148 * (X(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ X(7)) + -0.009168998396237136 * (Z(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.009168998396237136 * (Z(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.038477381564013806 * (Y(1) @ X(2) @ X(5) @ Y(6)) + 0.038477381564013806 * (Y(1) @ Y(2) @ X(5) @ X(6)) + 0.038477381564013806 * (X(1) @ X(2) @ Y(5) @ Y(6)) + -0.038477381564013806 * (X(1) @ Y(2) @ Y(5) @ X(6)) + 0.008254330656463974 * (Y(1) @ Z(2) @ Z(4) @ Y(5)) + 0.008254330656463974 * (X(1) @ Z(2) @ Z(4) @ X(5)) + -0.0038180782116323244 * (Y(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Y(7)) + -0.011010996943064903 * (Y(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Y(7)) + -0.007192918731432575 * (Y(1) @ Z(2) @ Y(3) @ X(5) @ Z(6) @ X(7)) + -0.007192918731432575 * (X(1) @ Z(2) @ X(3) @ Y(5) @ Z(6) @ Y(7)) + -0.011010996943064903 * (X(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ X(7)) + -0.0038180782116323244 * (X(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ X(7)) + 0.07121221471823151 * (Z(1) @ Z(4)) + -0.0021078400948430893 * (Y(1) @ Z(2) @ Z(3) @ Y(5)) + -0.0021078400948430893 * (X(1) @ Z(2) @ Z(3) @ X(5)) + -0.04567030029544638 * (Y(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ Y(3)) + -0.04567030029544638 * (Y(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ Y(3)) + -0.04567030029544638 * (X(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ X(3)) + -0.04567030029544638 * (X(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ X(3)) + 0.0029270939882668615 * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Y(7)) + 0.0029270939882668615 * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Y(7)) + 0.0029270939882668615 * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ X(7)) + 0.0029270939882668615 * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ X(7)) + 0.006229806418892229 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(7)) + 0.006229806418892229 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(7)) + 0.07417678038684464 * (Z(1) @ Z(6)) + 0.00915690040715909 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(6)) + 0.00915690040715909 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(6)) + 0.03449388714822958 * (Z(2) @ Z(4)) + 0.03312340473075566 * (Z(2) @ Z(6)) + 0.07171035452618708 * (Z(2) @ Z(3)) + 0.0016416397981023102 * (Y(2) @ Z(4) @ Z(5) @ Y(6)) + 0.0016416397981023102 * (X(2) @ Z(4) @ Z(5) @ X(6)) + 0.03757222564650857 * (Y(2) @ X(3) @ X(4) @ Y(5)) + -0.03757222564650857 * (Y(2) @ Y(3) @ X(4) @ X(5)) + -0.03757222564650857 * (X(2) @ X(3) @ Y(4) @ Y(5)) + 0.03757222564650857 * (X(2) @ Y(3) @ Y(4) @ X(5)) + 0.0016416397981023102 * (Z(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + 0.0016416397981023102 * (Z(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + 0.03875564485727539 * (Y(2) @ X(3) @ X(6) @ Y(7)) + -0.03875564485727539 * (Y(2) @ Y(3) @ X(6) @ X(7)) + -0.03875564485727539 * (X(2) @ X(3) @ Y(6) @ Y(7)) + 0.03875564485727539 * (X(2) @ Y(3) @ Y(6) @ X(7)) + -0.008355235459945029 * (Y(2) @ Z(3) @ Z(5) @ Y(6)) + -0.008355235459945029 * (X(2) @ Z(3) @ Z(5) @ X(6)) + 0.07206611279473815 * (Z(2) @ Z(5)) + 0.0019161259400428338 * (Y(2) @ Z(3) @ Z(4) @ Y(6)) + 0.0019161259400428338 * (X(2) @ Z(3) @ Z(4) @ X(6)) + -0.010271361399987862 * (Y(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ Y(4)) + -0.010271361399987862 * (Y(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ Y(4)) + -0.010271361399987862 * (X(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ X(4)) + -0.010271361399987862 * (X(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ X(4)) + 0.07187904958803104 * (Z(2) @ Z(7)) + -0.009445166905550164 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(7)) + -0.009445166905550164 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(7)) + 0.03449388714822958 * (Z(3) @ Z(5)) + 0.03312340473075566 * (Z(3) @ Z(7)) + 0.07206611279473815 * (Z(3) @ Z(4)) + 0.0019161259400428338 * (Y(3) @ Z(5) @ Z(6) @ Y(7)) + 0.0019161259400428338 * (X(3) @ Z(5) @ Z(6) @ X(7)) + -0.010271361399987862 * (Y(3) @ X(4) @ X(5) @ Y(6)) + 0.010271361399987862 * (Y(3) @ Y(4) @ X(5) @ X(6)) + 0.010271361399987862 * (X(3) @ X(4) @ Y(5) @ Y(6)) + -0.010271361399987862 * (X(3) @ Y(4) @ Y(5) @ X(6)) + -0.008355235459945029 * (Y(3) @ Z(4) @ Z(6) @ Y(7)) + -0.008355235459945029 * (X(3) @ Z(4) @ Z(6) @ X(7)) + 0.07187904958803104 * (Z(3) @ Z(6)) + -0.009445166905550166 * (Y(3) @ Z(4) @ Z(5) @ Y(7)) + -0.009445166905550166 * (X(3) @ Z(4) @ Z(5) @ X(7)) + 0.025940371141848058 * (Z(4) @ Z(6)) + 0.07247046642983263 * (Z(4) @ Z(5)) + 0.04625314440564194 * (Y(4) @ X(5) @ X(6) @ Y(7)) + -0.04625314440564194 * (Y(4) @ Y(5) @ X(6) @ X(7)) + -0.04625314440564194 * (X(4) @ X(5) @ Y(6) @ Y(7)) + 0.04625314440564194 * (X(4) @ Y(5) @ Y(6) @ X(7)) + 0.07219351554749001 * (Z(4) @ Z(7)) + 0.025940371141848058 * (Z(5) @ Z(7)) + 0.07219351554749001 * (Z(5) @ Z(6)) + 0.07526107314196621 * (Z(6) @ Z(7))
)

In [4]:
hf_state = np.array([1, 1, 1, 1, 0, 0, 0, 0])  # Example Hartree-Fock state
exc = [[3,7]]
qubits = 8
params = qml.numpy.array([np.pi/4], requires_grad=True)  # Example parameters, can be changed
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def newstate(params, ash_excitation, hf_state, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.DoubleExcitation(params, wires=excitation)
        elif len(ash_excitation[i]) == 2:
            qml.SingleExcitation(params, wires=excitation)
    return qml.expval(H)
@qml.qnode(dev)
def statek(params, ash_excitation, hf_state, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.DoubleExcitation(params, wires=excitation)
        elif len(ash_excitation[i]) == 2:
            qml.SingleExcitation(params, wires=excitation)
    return qml.state()

def c(params):
    cost = newstate(params, ash_excitation, hf_state, H)
    return cost

result = minimize(c, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

print('Optimized parameters:', result.x)
params = result.x
energy = c(params)

s = newstate(params, exc, hf_state, H)
print('Energy:', s)

sstate = statek(params, ash_excitation, hf_state, H)
print('State:', sstate)

Optimized parameters: [1.12299271]
Energy: [-1.31327953]
State: [[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.       

In [3]:

active_electrons = 4
qubits = 8
hf_state = np.array([1,1,1,1,0,0,0,0])  #Example HF state
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    return qml.expval(res)
@qml.qnode(dev)
def commutator_1(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    return qml.expval(res)
@qml.qnode(dev)
def state1(params, ash_excitation, hf_state, H):
    print('For state prep, using previous exc. ', ash_excitation) 
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #print('DE', ash_excitation[i])
            #print('Params', params[i])  
            qml.DoubleExcitation(params[i], wires = excitation)
        elif len(ash_excitation[i]) == 2:
            qml.SingleExcitation(params[i], wires = excitation)
    return qml.state()
@qml.qnode(dev)
def energy1(params, ash_excitation, hf_state, H):
    #print('ash_excitation', ash_excitation) 
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #print('DE', ash_excitation[i])
            #print('Params', params[i])  
            qml.DoubleExcitation(params[i], wires = excitation)
        elif len(ash_excitation[i]) == 2:
            qml.SingleExcitation(params[i], wires = excitation)
    return qml.expval(H)

def cost(params):
    energy = energy1(params, ash_excitation, hf_state, H)
    return energy

singles, doubles = qml.qchem.excitations(active_electrons, qubits)

op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
print('Total excitations are', len(operator_pool))
states = [hf_state]
ash_excitation = []
params = np.zeros(len(ash_excitation), requires_grad=True) 
print('parasm are',params)
    

for j in range(10):
    print('The adapt iteration now is', j)  #Adapt iteration
    max_value = float('-inf')
    max_operator = None
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
    #print('The value of current state is', k)  #Current state
    for i in operator_pool:
        #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
        w = qml.fermi.jordan_wigner(i)  #JW transformation
        if np.array_equal(k, hf_state): # If the current state is the HF state
            current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
        else:
            current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
        #print(f'The current value is', {i} ,'and',{current_value})  #Current value of the commutator
        if current_value > max_value:
            max_value = current_value
            max_operator = i
    print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
    old_grad.append(max_value)  #Appending the old gradient value
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
    params = np.append(params, 0.0)  #Parameters initialization
    #Energy calculation
    result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})
    print("Final updated parameters:", result.x)
    print("Final cost:", result.fun)
    params= (result.x)
    print('The params are', params)
    energies.append(result.fun)
    ashs = [ash_excitation[-1]]
    print('Ashs is', ashs)
    ostate = state1(params, ashs, hf_state, H)
    states.append(ostate)

Total excitations are 26
parasm are []
The adapt iteration now is 0
The highest operator value is 0.3100451588582031 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.1229927]
Final cost: -1.4300836708403746
The params are [1.1229927]
Ashs is [[2, 3, 6, 7]]
For state prep, using previous exc.  [[2, 3, 6, 7]]
The adapt iteration now is 1
The highest operator value is 0.36333417749699 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [1.0971619  1.37775801]
Final cost: -1.569460383552164
The params are [1.0971619  1.37775801]
Ashs is [[0, 3, 5, 6]]
For state prep, using previous exc.  [[0, 3, 5, 6]]
The adapt iteration now is 2
The highest operator value is 0.38830304131634474 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [ 1.09716186e+00  1.37775787e+00 -5.34148875e-08]
Final cost: -1.569460383552165
The params are [ 

In [6]:
energies
old_grad
ash_excitation

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [2, 3, 6, 7],
 [0, 3, 5, 6],
 [2, 3, 6, 7],
 [0, 3, 5, 6],
 [2, 3, 6, 7],
 [0, 3, 5, 6],
 [2, 3, 6, 7],
 [0, 3, 5, 6]]

## Multiple single det. with Beta for gradient

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
theta = np.pi/4
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

print('Using G1|HF> + G2|HF> + ... + Gn|HF> as the ansatz for gradient method')




def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()
    



    
    #@qml.qnode(dev1)
    def s2state_combined(hf_state, ashs, theta):
        """Create beta*(G1|HF⟩ + G2|HF⟩)"""
        super_states = []
        for i in range(len(ashs)):
            #params_copy = [0.0] * len(ashs)
            #params_copy[i] = params2[i]  # Only activate one excitation at a time

            # Create a separate mini-circuit for each excitation
            @qml.qnode(dev1)
            def tmp_qnode():
                [qml.PauliX(w) for w in np.nonzero(hf_state)[0]]
                #print('Excitations going to prepare s2 combined', ashs[i])
                if len(ashs[i]) == 4:
                    qml.DoubleExcitation(theta, wires=ashs[i])
                elif len(ashs[i]) == 2:
                    qml.SingleExcitation(theta, wires=ashs[i])
                return qml.state()
            
            state_i = tmp_qnode()
            super_states.append(state_i)

        summed_state = np.sum(super_states, axis=0)
        summed_state /= np.linalg.norm(summed_state)
        return summed_state

    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state_combined(hf_state, ashs, theta)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        #params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)] #3
        beta = x[len(ashs)+1]  #4
        return global_cost(params, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            if i not in operator_check:
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)
                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        print('Preparing s2 state with ashs:', ashs)
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)

        
        s2ostate = beta * s2state_combined(hf_state, ashs, theta)
        #print(qml.draw(s2state_combined, max_length=100)(hf_state,ashs,theta))
        x0 = np.concatenate([params, np.array([alpha]), np.array([beta])])

        #print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        #params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        alpha = result.x[len(ashs)]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 1]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state_combined(hf_state, ashs, theta)
        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        final_state = s2state_combined(hf_state, ashs, theta)
        counts = measure(final_state)
        print('Measurement of final state before going to next grad:', counts)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        #print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        print('\n')
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using G1|HF> + G2|HF> + ... + Gn|HF> as the ansatz for gradient method
H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582031 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Preparing s2 state with ashs: [[2, 3, 6, 7]]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.4300836708312177
        x: [3.22407893+0.j  0.33711562+0.8j 1.74123946+0.8j]
      nit: 14
      jac: [-1.639e-07 -1.639e-06  3.129e-07]
 hess_inv

KeyboardInterrupt: 